In [100]:
import os
import subprocess
import re
import time

# --- Setup Paths (Ensure these environment variables are correctly set in your environment) ---
mopmc_home = os.getenv('MOPMC_HOME')
storm_home = os.getenv('STORM_HOME')
prism_home = os.getenv('PRISM_HOME')

input_folder = os.path.join(os.getcwd(), 'obj_num_eval_input')
d_n_r_model = os.path.join(input_folder, 'dive_and_rise_action_rewards.nm')
d_n_r_property_list = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.props')]

# Storm can handles number of objectives up to 9
def check_id(f, threshold=10) :
    match = re.search(r'dive_and_rise_prop_(\d+)\.props', os.path.basename(f)) 
    if match:
            prop_id = int(match.group(1)) 
    else:
        print(f"Warning: Could not extract integer ID from prop file: {f}")
    if prop_id < threshold:
        return True
    else:
        return False
            
d_n_r_property_list_below_10 = [f for f in d_n_r_property_list if check_id(f)]
d_n_r_property_list_above_incl_10 = [f for f in d_n_r_property_list if not check_id(f)]


d_n_r_property_10= os.path.join(input_folder, 'dive_and_rise_prop_10.props')
d_n_r_property_5= os.path.join(input_folder, 'dive_and_rise_prop_5.props')
d_n_r_property_6= os.path.join(input_folder, 'dive_and_rise_prop_6.props')
d_n_r_property_7= os.path.join(input_folder, 'dive_and_rise_prop_7.props')
d_n_r_property_8= os.path.join(input_folder, 'dive_and_rise_prop_8.props')
d_n_r_property_9= os.path.join(input_folder, 'dive_and_rise_prop_9.props')
d_n_r_property_60= os.path.join(input_folder, 'dive_and_rise_prop_60.props')
d_n_r_property_80= os.path.join(input_folder, 'dive_and_rise_prop_80.props')
d_n_r_property_100= os.path.join(input_folder, 'dive_and_rise_prop_100.props')

# Construct full executable paths with checks
prism_executable = os.path.join(prism_home, 'bin', 'prism') if prism_home else None
storm_executable = os.path.join(storm_home, 'build', 'bin', 'storm') if storm_home else None
mopmc_executable = os.path.join(mopmc_home, 'build', 'mopmc') if mopmc_home else None

print(f"input folder: {input_folder}")
print(d_n_r_property_list_above_incl_10)
#print(d_n_r_model)

input folder: /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input
['/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_30.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_90.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_70.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_100.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_80.props', '/home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_50.props', '/home/guoxin/PycharmProjects/r

In [ ]:
prism_command = f"{prism_executable} {d_n_r_model} {d_n_r_property_100}"
! $prism_command

In [ ]:
storm_command = f"{storm_executable} --prism {d_n_r_model} --prop {d_n_r_property_9}"
! $storm_command

In [ ]:
mopmc_command = f"{mopmc_executable} -m {d_n_r_model} -p {d_n_r_property_80} -q achievability"
! $mopmc_command

In [ ]:
mopmc_command = f"{mopmc_executable} -m {d_n_r_model} -p {d_n_r_property_60} -q achievability -v standard"
! $mopmc_command

In [101]:
# --- Define the parsing function ---
def parse_output(output_string, tool_name, query):
    """
    Parses the output of the PRISM tool and extracts relevant metrics.
    """
    extracted_data = {}
    states_match = re.search(r"States:\s*(\d+)", output_string, re.IGNORECASE)
    transitions_match = re.search(r"Transitions:\s*(\d+)", output_string, re.IGNORECASE)
    choices_match = re.search(r"Choices:\s*(\d+)", output_string, re.IGNORECASE)
    loop_count_match = re.search(r"MOPMC main loop count:\s*(\d+)", output_string, re.IGNORECASE)
    loop_count_key = query + '_loop_count_' + tool_name

    if states_match:
        extracted_data['States'] = int(states_match.group(1))
    if choices_match:
        extracted_data['Choices'] = int(choices_match.group(1))
    if transitions_match:
        extracted_data['Transitions'] = int(transitions_match.group(1))
    if loop_count_match:
        extracted_data[loop_count_key] = int(loop_count_match.group(1))
    return extracted_data

In [102]:
def run_tool_and_parse(command_list, parser_function=None, tool_name="Tool", query=None):
    """
    Executes a command and, optionally, parses its output using a provided function.

    Args:
        command_list (list): The command and its arguments as a list.
        tool_name (str): A descriptive name for the tool (e.g., "PRISM", "Storm").
        parser_function (callable, optional): A function that takes the tool's
                                               stdout (string) and returns a dict of extracted data.
                                               If None, no parsing is performed.

    Returns:
        dict: A dictionary containing extracted data and a 'success' flag.
              Returns an empty dictionary if the executable is not found or an error occurs.
    """
    extracted_data = {}

    if not command_list or not command_list[0] or not os.path.exists(command_list[0]):
        print(f"Error: {tool_name} executable not found or command list is invalid. "
              f"Attempted executable: {command_list[0] if command_list else 'None'}")
        return extracted_data

    try:
        print(f"Running {tool_name} command: {' '.join(command_list)}")
        result = subprocess.run(
            command_list,
            capture_output=True,
            text=True,
            check=True
        )
        tool_output = result.stdout
        if parser_function:
            parsed_info = parser_function(tool_output, tool_name, query)
            extracted_data.update(parsed_info) # Add parsed info to the result
    except Exception as e:
        print(f"An unexpected error occurred while running {tool_name}: {e}")

    return extracted_data



In [103]:
def generate_aq_command_list(executable, model, prop_list, tool_name, query='aq'):
    command_list = []
    if query!='aq':
        raise ValueError(f"Unsupported query: {query}!")
    for prop in prop_list:
        if tool_name == 'mopmc':
            command = [executable, '-m', model, '-p', prop, '-q', 'achievability']            
        elif tool_name == "mopmc_cpu":
            command = [executable, '-m', model, '-p', prop, '-q', 'achievability', '-v', 'standard']
        elif tool_name == "storm":
            command = [executable, '--prism', model, '--prop', prop]
        elif tool_name == "prism":        
            command = [executable, model, prop]
        else:
             raise ValueError(f"Incorrect tool name for achievability query: {tool_name}!")
        command_list.append(command)
    return command_list


def generate_cq_command_list(executable, model, prop_list, tool_name, query='ccq'):
    command_list = []
    
    for prop in prop_list:
        if query=='ccq':
            if tool_name == 'mopmc':
                command = [executable, '-m', model, '-p', prop, '-q', 'convex']            
            elif tool_name == "mopmc_cpu":
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-v', 'standard']
            else:
                 raise ValueError(f"Incorrect tool name for convex query: {tool_name}!")
        elif query=='ucq':    
            if tool_name == 'mopmc':
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-c', 'n']            
            elif tool_name == "mopmc_cpu":
                command = [executable, '-m', model, '-p', prop, '-q', 'convex', '-c', 'n', '-v', 'standard']
            else:
                 raise ValueError(f"Incorrect tool name for convex query: {tool_name}!")
        else:
            raise ValueError(f"Unsupported query: {query}!")
        command_list.append(command)
    return command_list



In [104]:
def run_experiment_2(executable, model, prop_list, tool_name, query='aq'):
    results = []
    if query=='aq':
        command_list = generate_aq_command_list(executable, model, prop_list, tool_name, query)
    else:
        command_list = generate_cq_command_list(executable, model, prop_list, tool_name, query)
    runtime_key = query + '_run_time_' + tool_name   
    for prop, command in zip(prop_list, command_list):                    
        start_time = time.perf_counter()
        data = run_tool_and_parse(command, parse_output, tool_name, query)
        end_time = time.perf_counter()
        data[runtime_key] = end_time - start_time        
        match = re.search(r'dive_and_rise_prop_(\d+)\.props', os.path.basename(prop)) 
        prop_id = None
        if match:
            prop_id = int(match.group(1)) 
        else:
            print(f"Warning: Could not extract integer ID from prop file: {prop}")              
        data['obj_num'] = prop_id
        results.append(data)
    return results

In [95]:
mopmc_gpu_aq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc", "aq")
mopmc_gpu_aq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-GPU Achievability Query Results:")
print(mopmc_gpu_aq_results)

Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props -q achievability
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props -q achievability
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3

In [96]:
mopmc_cpu_aq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc_cpu", "aq")
mopmc_cpu_aq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-CPU Achievability Query Results:")
print(mopmc_cpu_aq_results)

Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props -q achievability -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj

In [97]:
mopmc_gpu_ccq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc", "ccq")
mopmc_gpu_ccq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-GPU Convex Query Results:")
print(mopmc_gpu_ccq_results)

Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props -q convex
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props -q convex
Running mopmc command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_30.props -q con

In [105]:
mopmc_cpu_ccq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc_cpu", "ccq")
mopmc_cpu_ccq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-CPU Convex Query Results:")
print(mopmc_cpu_ccq_results)

Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props -q convex -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props -q convex -v standard
Running mopmc_cpu command: /home/guoxin/CLionProjects/mopmc-dev-v3/build/mopmc -m /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm -p /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_inpu

In [ ]:
mopmc_gpu_ucq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc", "ucq")
mopmc_gpu_ucq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-GPU (Unconstrained) Convex Query Results:")
print(mopmc_gpu_ucq_results)

In [ ]:
mopmc_cpu_ucq_results= run_experiment_2(mopmc_executable, d_n_r_model, d_n_r_property_list, "mopmc_cpu", "ucq")
mopmc_cpu_ucq_results.sort(key= lambda item: item['obj_num'])
print(f"\nMOPMC-CPU (Unconstrained) Convex Query Results:")
print(mopmc_cpu_ucq_results)

In [67]:
# for storm, run up to objective number < 10
storm_results= run_experiment_2(storm_executable, d_n_r_model, d_n_r_property_list_below_10, "storm")
storm_results.sort(key= lambda item: item['obj_num'])
print(f"\nStorm Results:")
print(storm_results)

Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_8.props
Running storm command: /home/guoxin/Downloads/storm/build/bin/storm --prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm --prop /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_9.props
Running storm command:

In [68]:
prism_results = run_experiment_2(prism_executable, d_n_r_model, d_n_r_property_list, "prism")
prism_results.sort(key= lambda item: item['obj_num'])
print(f"\nPRISM Results:")
print(prism_results)

Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_3.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_40.props
Running prism command: /home/guoxin/Downloads/prism-4.4-linux64/bin/prism /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_action_rewards.nm /home/guoxin/PycharmProjects/realtime-drl-switch/scripts/tool_evaluation/obj_num_eval_input/dive_and_rise_prop_30.props
Running prism command: /home/guoxin/Downloads/p

In [106]:
from itertools import zip_longest
combined_results= []

for d0, d1, d2, d3, d4, d5 in zip_longest(storm_results, prism_results, 
                                          mopmc_gpu_aq_results, mopmc_cpu_aq_results,
                                          mopmc_gpu_ccq_results, mopmc_cpu_ccq_results,
                                          fillvalue={}):
    combined_dict = {}
    combined_dict.update(d0)
    combined_dict.update(d1)
    combined_dict.update(d2)
    combined_dict.update(d3)
    combined_dict.update(d4)
    combined_dict.update(d5)
    combined_results.append(combined_dict)

import pandas as pd
combined_results_df = pd.DataFrame(combined_results)
print(f"current column names: {combined_results_df.columns}")
combined_results_df['ccq_run_time_mopmc_per_it'] = combined_results_df['ccq_run_time_mopmc'] / combined_results_df['ccq_loop_count_mopmc']
combined_results_df['ccq_run_time_mopmc_cpu_per_it'] = combined_results_df['ccq_run_time_mopmc_cpu'] / combined_results_df['ccq_loop_count_mopmc_cpu']
new_column_order = ['obj_num', 'States', 'Choices', 'Transitions', 
                    'aq_run_time_storm', 'aq_run_time_prism', 
                    'aq_run_time_mopmc', 'aq_run_time_mopmc_cpu', 
                    'ccq_run_time_mopmc', 'ccq_loop_count_mopmc', 'ccq_run_time_mopmc_per_it', 
                    'ccq_run_time_mopmc_cpu', 'ccq_loop_count_mopmc_cpu', 'ccq_run_time_mopmc_cpu_per_it']
combined_results_df = combined_results_df[new_column_order]

num_columns = ['aq_run_time_storm', 'aq_run_time_prism', 'aq_run_time_mopmc',
       'aq_run_time_mopmc_cpu', 'ccq_run_time_mopmc',
       'ccq_run_time_mopmc_cpu', 'ccq_run_time_mopmc_per_it', 'ccq_run_time_mopmc_cpu_per_it']
combined_results_df[num_columns] = combined_results_df[num_columns].round(2)
print(combined_results_df)

current column names: Index(['States', 'Choices', 'Transitions', 'aq_run_time_storm', 'obj_num',
       'aq_run_time_prism', 'aq_loop_count_mopmc', 'aq_run_time_mopmc',
       'aq_loop_count_mopmc_cpu', 'aq_run_time_mopmc_cpu',
       'ccq_loop_count_mopmc', 'ccq_run_time_mopmc',
       'ccq_loop_count_mopmc_cpu', 'ccq_run_time_mopmc_cpu'],
      dtype='object')
    obj_num  States  Choices  Transitions  aq_run_time_storm  \
0         2   15051    30001        89203               0.11   
1         3   15051    30001        89203               0.23   
2         4   15051    30001        89203               0.20   
3         5   15051    30001        89203               0.38   
4         6   15051    30001        89203               0.81   
5         7   15051    30001        89203               3.94   
6         8   15051    30001        89203              24.63   
7         9   15051    30001        89203             150.78   
8        10   15051    30001        89203                Na